In [1]:
import pandas as pd
from datetime import datetime,date 
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split as tts
from sklearn.tree import DecisionTreeRegressor
import pickle
import json 

In [2]:
df1=pd.read_csv("Scraped_dataset.csv")

In [3]:
df1.head()

,Date of Booking,Date of Journey,Airline-Class,Departure Time,Arrival Time,Duration,Total Stops,Price
0,15/01/2023,16/01/2023,SpiceJet \nSG-8169\nECONOMY,20:00\nDelhi,22:05\nMumbai,02h 05m,non-stop,"5,335"
1,15/01/2023,16/01/2023,Indigo \n6E-2519\nECONOMY,23:00\nDelhi,01:20\nMumbai,02h 20m,non-stop,"5,899"
2,15/01/2023,16/01/2023,GO FIRST \nG8- 354\nECONOMY,22:30\nDelhi,00:40\nMumbai,02h 10m,non-stop,"5,801"
3,15/01/2023,16/01/2023,SpiceJet \nSG-8709\nECONOMY,18:50\nDelhi,20:55\nMumbai,02h 05m,non-stop,"5,794"
4,15/01/2023,16/01/2023,Air India \nAI-805\nECONOMY,20:00\nDelhi,22:10\nMumbai,02h 10m,non-stop,"5,955"


In [4]:
def split(data):
    airline=[]
    Class=[]
    for i in data:
        temp=i.split("\n")
        airline.append(temp[0])
        Class.append(temp[-1])
    return pd.DataFrame({"airline":airline,"Class":Class})

temp1=split(df1["Airline-Class"])
df2=pd.concat([df1,temp1],axis=1)


In [5]:
df2.head()

,Date of Booking,Date of Journey,Airline-Class,Departure Time,Arrival Time,Duration,Total Stops,Price,airline,Class
0,15/01/2023,16/01/2023,SpiceJet \nSG-8169\nECONOMY,20:00\nDelhi,22:05\nMumbai,02h 05m,non-stop,"5,335",SpiceJet,ECONOMY
1,15/01/2023,16/01/2023,Indigo \n6E-2519\nECONOMY,23:00\nDelhi,01:20\nMumbai,02h 20m,non-stop,"5,899",Indigo,ECONOMY
2,15/01/2023,16/01/2023,GO FIRST \nG8- 354\nECONOMY,22:30\nDelhi,00:40\nMumbai,02h 10m,non-stop,"5,801",GO FIRST,ECONOMY
3,15/01/2023,16/01/2023,SpiceJet \nSG-8709\nECONOMY,18:50\nDelhi,20:55\nMumbai,02h 05m,non-stop,"5,794",SpiceJet,ECONOMY
4,15/01/2023,16/01/2023,Air India \nAI-805\nECONOMY,20:00\nDelhi,22:10\nMumbai,02h 10m,non-stop,"5,955",Air India,ECONOMY


In [6]:
temp2=[]
for i in range(len(df2)):
    temp2.append(int(df2["Price"][i].replace(",","")))
df2["new_price"]=pd.Series(temp2)
df3=df2.drop(["Price","Airline-Class"],axis=1)

In [7]:
df3.head()

,Date of Booking,Date of Journey,Departure Time,Arrival Time,Duration,Total Stops,airline,Class,new_price
0,15/01/2023,16/01/2023,20:00\nDelhi,22:05\nMumbai,02h 05m,non-stop,SpiceJet,ECONOMY,5335
1,15/01/2023,16/01/2023,23:00\nDelhi,01:20\nMumbai,02h 20m,non-stop,Indigo,ECONOMY,5899
2,15/01/2023,16/01/2023,22:30\nDelhi,00:40\nMumbai,02h 10m,non-stop,GO FIRST,ECONOMY,5801
3,15/01/2023,16/01/2023,18:50\nDelhi,20:55\nMumbai,02h 05m,non-stop,SpiceJet,ECONOMY,5794
4,15/01/2023,16/01/2023,20:00\nDelhi,22:10\nMumbai,02h 10m,non-stop,Air India,ECONOMY,5955


In [8]:
def split_loc(data):
    out=[]
    for i in data:
        temp=i.split("\n")
        out.append(temp[-1])
    return pd.Series(out)


destination=pd.DataFrame({"destination":split_loc(df3['Arrival Time'])})
departure=pd.DataFrame({"departure":split_loc(df3['Departure Time'])})
df4=pd.concat([df3,destination,departure],axis=1)

In [9]:
df4.head()

,Date of Booking,Date of Journey,Departure Time,Arrival Time,Duration,Total Stops,airline,Class,new_price,destination,departure
0,15/01/2023,16/01/2023,20:00\nDelhi,22:05\nMumbai,02h 05m,non-stop,SpiceJet,ECONOMY,5335,Mumbai,Delhi
1,15/01/2023,16/01/2023,23:00\nDelhi,01:20\nMumbai,02h 20m,non-stop,Indigo,ECONOMY,5899,Mumbai,Delhi
2,15/01/2023,16/01/2023,22:30\nDelhi,00:40\nMumbai,02h 10m,non-stop,GO FIRST,ECONOMY,5801,Mumbai,Delhi
3,15/01/2023,16/01/2023,18:50\nDelhi,20:55\nMumbai,02h 05m,non-stop,SpiceJet,ECONOMY,5794,Mumbai,Delhi
4,15/01/2023,16/01/2023,20:00\nDelhi,22:10\nMumbai,02h 10m,non-stop,Air India,ECONOMY,5955,Mumbai,Delhi


In [10]:
def split_loc(data):
    out=[]
    for i in data:
        temp=i.split(":")[0]
        temp=int(temp)
        if(temp>=3 and temp<=12):
             out.append("morning")
        elif(temp>12 and temp<=18):
             out.append("afternoon")
        else:
             out.append("night")
    return out
boarding_time=pd.DataFrame(split_loc(df4["Departure Time"]))
df4["boarding_time"]=boarding_time
df4.drop("Departure Time",axis=1,inplace=True)

In [12]:
def str_date(data):
    temp=[]
    for i in data:
        date_string=i.replace("/","")
        date_object = datetime.strptime(date_string, "%d%m%Y").date()
        temp.append(date_object)
    return temp
    
def numOfDays(date1, date2):
    if date2 > date1:
        return (date2 - date1).days
    else:
        return (date1 - date2).days
df4["booking"]=pd.Series(str_date(df4['Date of Booking']))
df4['journey']=pd.Series(str_date(df4['Date of Journey']))


temp3=[]
for i in range(len(df4)):
    value=numOfDays(df4.booking[i],df4.journey[i])
    temp3.append(value)
df4["days"]=pd.Series(temp3)


In [13]:
df5=df4.drop(["Date of Booking","Date of Journey","Arrival Time"],axis=1)

In [14]:
df5.head()

,Duration,Total Stops,airline,Class,new_price,destination,departure,boarding_time,booking,journey,days
0,02h 05m,non-stop,SpiceJet,ECONOMY,5335,Mumbai,Delhi,night,2023-01-15,2023-01-16,1
1,02h 20m,non-stop,Indigo,ECONOMY,5899,Mumbai,Delhi,night,2023-01-15,2023-01-16,1
2,02h 10m,non-stop,GO FIRST,ECONOMY,5801,Mumbai,Delhi,night,2023-01-15,2023-01-16,1
3,02h 05m,non-stop,SpiceJet,ECONOMY,5794,Mumbai,Delhi,afternoon,2023-01-15,2023-01-16,1
4,02h 10m,non-stop,Air India,ECONOMY,5955,Mumbai,Delhi,night,2023-01-15,2023-01-16,1


In [15]:
def hours(data):
    temp=[]
    for i in data:
        temp.append(int(i.split("h")[0]))
    return pd.Series(temp)
flying_hours=hours(df5.Duration)
df5["flying_hours"]=flying_hours

In [16]:
df6=df5.drop(["Duration","booking","journey"],axis=1)

In [18]:
print("before changing number of stops:")
print(df6['Total Stops'].unique())
c=[i for i in df6['Total Stops'].unique() if i not in ['non-stop', '1-stop', '2+-stop'] ]
for i in c:
    df6['Total Stops'].replace(i,'1-stop',inplace=True)
print("after changing number of stops:")
df6['Total Stops'].unique()

before changing number of stops:
['non-stop' '1-stop' '2+-stop'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Indore'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia IDR'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia DIB'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Guwahati'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia VTZ'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia RPR'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Mumbai'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Delhi'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Pune'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia PNQ']
after changing number of stops:


array(['non-stop', '1-stop', '2+-stop'], dtype=object)

In [19]:
df6.head()

,Total Stops,airline,Class,new_price,destination,departure,boarding_time,days,flying_hours
0,non-stop,SpiceJet,ECONOMY,5335,Mumbai,Delhi,night,1,2
1,non-stop,Indigo,ECONOMY,5899,Mumbai,Delhi,night,1,2
2,non-stop,GO FIRST,ECONOMY,5801,Mumbai,Delhi,night,1,2
3,non-stop,SpiceJet,ECONOMY,5794,Mumbai,Delhi,afternoon,1,2
4,non-stop,Air India,ECONOMY,5955,Mumbai,Delhi,night,1,2


In [20]:
df6["price"]=df6['new_price']
df6.drop("new_price",axis=1,inplace=True)

In [21]:
df6.head()

,Total Stops,airline,Class,destination,departure,boarding_time,days,flying_hours,price
0,non-stop,SpiceJet,ECONOMY,Mumbai,Delhi,night,1,2,5335
1,non-stop,Indigo,ECONOMY,Mumbai,Delhi,night,1,2,5899
2,non-stop,GO FIRST,ECONOMY,Mumbai,Delhi,night,1,2,5801
3,non-stop,SpiceJet,ECONOMY,Mumbai,Delhi,afternoon,1,2,5794
4,non-stop,Air India,ECONOMY,Mumbai,Delhi,night,1,2,5955


In [22]:
le= LabelEncoder()
df6["stops"]=abs(le.fit_transform(df6['Total Stops'])-2)

In [23]:
df6.stops.replace(2,4,inplace=True)
df6.stops.replace(1,2,inplace=True)
df6.stops.replace(4,1,inplace=True)

In [24]:
df7=df6.drop("Total Stops",axis=1)

In [25]:
df7.head()

,airline,Class,destination,departure,boarding_time,days,flying_hours,price,stops
0,SpiceJet,ECONOMY,Mumbai,Delhi,night,1,2,5335,0
1,Indigo,ECONOMY,Mumbai,Delhi,night,1,2,5899,0
2,GO FIRST,ECONOMY,Mumbai,Delhi,night,1,2,5801,0
3,SpiceJet,ECONOMY,Mumbai,Delhi,afternoon,1,2,5794,0
4,Air India,ECONOMY,Mumbai,Delhi,night,1,2,5955,0


In [26]:
df7.airline=le.fit_transform(df7.airline)
df7.Class=le.fit_transform(df7.Class)
df7.destination=le.fit_transform(df7.destination)
df7.departure=le.fit_transform(df7.departure)
df7.boarding_time=le.fit_transform(df7.boarding_time)

In [28]:
price=df7.price
final_df=df7.drop(["price",'flying_hours',"boarding_time"],axis=1)

In [29]:
final_df.head()

,airline,Class,destination,departure,days,stops
0,6,1,6,3,1,0
1,5,1,6,3,1,0
2,4,1,6,3,1,0
3,6,1,6,3,1,0
4,0,1,6,3,1,0


In [31]:
x_train,x_test,y_train,y_test=tts(final_df,price,test_size=0.2)

In [32]:
# Create a decision tree regression model
dt = DecisionTreeRegressor()
# Fit the model to the data
dt.fit(x_train, y_train)

DecisionTreeRegressor()

In [33]:
dt.score(x_test,y_test)

0.9090895123728298

In [34]:
with open("flight_price_model.pickle","wb") as f:
    pickle.dump(dt,f)

In [35]:
columns={'data_columns': [col.lower() for col in x_train.columns]}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))

In [36]:
print(final_df.departure.unique())
print(df6.departure.unique())

[3 6 1 4 5 2 0]
['Delhi' 'Mumbai' 'Bangalore' 'Hyderabad' 'Kolkata' 'Chennai' 'Ahmedabad']


In [37]:
locations=['Ahmedabad','Bangalore','Chennai','Delhi','Hyderabad','Kolkata','Mumbai']
location={"location":locations}
with open("locations.json","w") as f:
    f.write(json.dumps(location))

In [38]:
print(final_df.airline.unique())
print(df6.airline.unique())

[6 5 4 0 1 8 2 3 7]
['SpiceJet ' 'Indigo ' 'GO FIRST ' 'Air India ' 'AirAsia ' 'Vistara '
 'AkasaAir ' 'AllianceAir ' 'StarAir ']


In [39]:
airlines=['Air India','AirAsia','AkasaAir','AllianceAir','GO FIRST','Indigo','SpiceJet','StarAir','Vistara']
airline={"airline":airlines}
with open("airline.json","w") as f:
    f.write(json.dumps(airline))

In [40]:
print(final_df.Class.unique())
print(df6.Class.unique())

[1 3 0 2]
['ECONOMY' 'PREMIUMECONOMY' 'BUSINESS' 'FIRST']


In [41]:
Classes=['BUSINESS','ECONOMY','FIRST','FIRST']
Class={"Class":Classes}
with open("Class.json","w") as f:
    f.write(json.dumps(Class))